In [1]:
#!pip install -U openpyxl

In [2]:
import os
import re
import pandas as pd
import numpy as np
from datetime import date, datetime
from openpyxl import load_workbook
import warnings
warnings.filterwarnings('ignore')

In [3]:
dh_inicio = datetime.today()

### CARGA DOS ARQUIVOS DOS CICLOS

In [4]:
# Caminho da pasta dos arquivos
caminho_pasta = r"C:\Projetos\PPSI\BASE\CICLOS-1-2-3-4"

# Lista os arquivos da pasta
arquivos = os.listdir(caminho_pasta)

# Filtra apenas arquivos (ignora subpastas)
arquivos = [f for f in arquivos if os.path.isfile(os.path.join(caminho_pasta, f))]

# Cria um DataFrame com os nomes
df = pd.DataFrame(arquivos, columns=["nome_arquivo"])

# Salva em Excel
#df.to_excel(r"C:\Users\MGI\Documents\COMAT\REFERENCIAS\CICLOS-1-2-3\CICLO_1_2_3_arquivos.xlsx", index=False)

print("Lista de arquivos salva com sucesso!")

Lista de arquivos salva com sucesso!


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   nome_arquivo  627 non-null    object
dtypes: object(1)
memory usage: 5.0+ KB


### FORM3GERAL-Plano-Trabalho_CONSOLIDADO

In [6]:

# Lista para armazenar os DataFrames da aba FORM2.4
dados_form3geral = []

# Lista para armazenar o resumo das contagens
resumo_linhas_antes = []

for arquivo in arquivos:
    caminho_arquivo = os.path.join(caminho_pasta, arquivo)

    nome_arquivo = os.path.basename(caminho_arquivo)
    total_linhas_arquivo = None
    linhas_totais = 0
    linhas_lidas = 0

    try:

        # 1. Contar linhas totais no Excel
        wb_contagem = load_workbook(filename=caminho_arquivo, read_only=True)
        ws_contagem = wb_contagem['FORM3GERAL']
        total_linhas_arquivo = ws_contagem.max_row
        
        # Lê valores da visão macro (H9 e I9)
        wb = load_workbook(caminho_arquivo, data_only=True)
        ws = wb['FORM3GERAL']
        qtd_medidas = ws['D5'].value
        qtd_prioridades = ws['F5'].value

        # Lê os dados a partir da 6ª linha (header=5)
#        df = pd.read_excel(caminho_arquivo, sheet_name='FORM3GERAL', header=5, engine='openpyxl')
        # 3. Lê os dados a partir da 6ª linha (header=5)
        df = pd.read_excel(caminho_arquivo, sheet_name='FORM3GERAL', 
                            dtype={'ID': object},     
                            keep_default_na=False,             
                            na_filter=False,
                             header = 5,
                            engine='openpyxl')

        df = df.iloc[:310]   # apenas linhas de 0 a 316

#        # Substitui strings vazias, espaços, tabs, etc., por NaN
#        df.replace(r'^\s*$', pd.NA, regex=True, inplace=True)      
#        df = df.dropna(how='all')
        
        # Registrar total inicial
        linhas_totais = len(df)
        
        # Registrar total final e linhas excluídas
        linhas_lidas = len(df)
        
        # Renomeia as colunas com base nas posições
        novos_nomes = {
            df.columns[1]: 'CICLO',           
            df.columns[2]: 'ID',
            df.columns[3]: 'Medida',
            df.columns[4]: 'Resposta das Medidas',
            df.columns[5]: 'Encaminhamento interno',
            df.columns[6]: 'Responsáveis',
            df.columns[7]: 'Departamento',
            df.columns[8]: 'Observação do Órgao para SGD',
            df.columns[9]: 'Previsão de Início',
            df.columns[10]: 'Previsão de Fim',
            df.columns[11]: 'Status Plano de Ação',
            df.columns[12]: 'Status Medida',
            df.columns[13]: 'Nova resposta',
            df.columns[14]: 'Prioridade'
        }
        df.rename(columns=novos_nomes, inplace=True)

        # Remove linhas com ID ausente
        df = df[df['ID'].notna()]
        df['ID'] = df['ID'].astype(str)
        df['Medida'] = df['Medida'].astype(str)
        # Colunas auxiliares
        df['Nome da Origem.2'] = arquivo
        df['Nome da Origem.1'] = df['Nome da Origem.2'].str.extract(r'-(.*?)-')
        #df['QTD TOTAL DE MEDIDAS'] = qtd_medidas
        #df['QTD MEDIDAS COM PRIORIDADE'] = qtd_prioridades
        df['Ciclo_Entrega'] = df['Nome da Origem.2'].str[:2]
        #df['ORGAO'] = df['Nome da Origem.2'].str.extract(r'(-.*?-)')
        
#        # Criação da nova coluna ID_MEDIDA
#        df['ID_MEDIDA'] = df['RESPOSTA DAS MEDIDAS'].map(mapa_resp_medida)
#        df['ID_MEDIDA'] = df['ID_MEDIDA'].apply(lambda x: str(int(x)) if pd.notnull(x) else None)

        # Seleção de colunas
        colunas = [
            'CICLO',
            'ID',
            'Medida',
            'Resposta das Medidas',
            'Encaminhamento interno',
            'Responsáveis',
            'Departamento',
            'Observação do Órgao para SGD',
            'Previsão de Início',
            'Previsão de Fim',
            'Status Plano de Ação',
            'Status Medida',
            'Nova resposta',
            'Prioridade',
#            'QTD TOTAL DE MEDIDAS',
#            'QTD MEDIDAS COM PRIORIDADE',
            'Nome da Origem.1',
            'Nome da Origem.2',
            'Ciclo_Entrega'
#            'ORGAO'
        ]
        df = df[colunas]

        dados_form3geral.append(df)
        #print(f"✅ Leitura concluída com sucesso: {arquivo}")

    except Exception as e:
        print(f"❌ Erro ao ler 'FORM3GERAL' em {arquivo}: {e}")

# 5. Registrar contagem no resumo antes da consolidação
    finally:
        resumo_linhas_antes.append({
            'arquivo': nome_arquivo,
            'total_linhas_no_arquivo': total_linhas_arquivo,
            'linhas_totais_lidas': linhas_totais,
            'linhas_lidas': linhas_lidas
        })


# Consolida tudo em um único DataFrame
df_form3geral = pd.concat(dados_form3geral, ignore_index=True)
#


# 7. DataFrame com o resumo ANTES da consolidação
df_resumo_antes = pd.DataFrame(resumo_linhas_antes)

# 8. DataFrame com o resumo APÓS a consolidação
df_resumo_apos = df_form3geral.groupby('Nome da Origem.2').size().reset_index(name='linhas_lidas')

# Renomear coluna para permitir o merge
df_resumo_apos = df_resumo_apos.rename(columns={'Nome da Origem.2': 'arquivo'})

# Merge dos dois resumos
df_resumo = pd.merge(df_resumo_antes, df_resumo_apos, on='arquivo', how='outer')

# Reorganizar colunas
colunas = [
    'arquivo',
    'total_linhas_no_arquivo',
    'linhas_totais_lidas',
    'linhas_lidas_x',  # Do antes
    'linhas_lidas_y',  # Do após
    'nome_de_origem.2_distintos'
]

# Renomear colunas para algo mais intuitivo
df_resumo.rename(columns={
    'linhas_lidas_x': 'linhas_lidas_antes',
    'linhas_lidas_y': 'linhas_lidas_apos'
}, inplace=True)

# Preencher NaNs com 0 (se houver erros na leitura de arquivos)
df_resumo.fillna(0, inplace=True)


# Exibir total de arquivos distintos processados
total_distintos = df_form3geral['Nome da Origem.2'].nunique()


total_arquivos_lidos = df_resumo_antes['linhas_lidas'].gt(0).sum()
print(f"📥 Total de arquivos lidos com sucesso: {total_arquivos_lidos}")

total_arquivos_consolidados = df_form3geral['Nome da Origem.2'].nunique()
print(f"📦 Total de arquivos únicos no consolidado (Nome da Origem.2): {total_arquivos_consolidados}")

df_resumo['total_arquivos_lidos'] = total_arquivos_lidos
df_resumo['total_arquivos_consolidados'] = total_arquivos_consolidados


# Resumo da execução
total_arquivos = len(arquivos)
sucesso = len(dados_form3geral)
erros = total_arquivos - sucesso

print("\n📊 Resumo da execução:")
print(f"🗂️ Total de arquivos considerados: {total_arquivos}")
print(f"✅ Arquivos lidos com sucesso: {sucesso}")
print(f"❌ Arquivos com erro: {erros}")
print("✅ Consolidação concluída com sucesso!")

❌ Erro ao ler 'FORM3GERAL' em C1-FUNDAJ-PlanilhaSISP-FUNDAJ.xlsx: expected <class 'openpyxl.worksheet.cell_range.MultiCellRange'>
❌ Erro ao ler 'FORM3GERAL' em C1-IFCE-IFCE - ferramenta_frameworkpsi_pt-3-basica-sem-macros-ciclo-1.xlsx: expected <class 'openpyxl.worksheet.cell_range.MultiCellRange'>
❌ Erro ao ler 'FORM3GERAL' em C1-UFAPE-PPSI-UFAPE.xlsx: expected <class 'openpyxl.worksheet.cell_range.MultiCellRange'>
❌ Erro ao ler 'FORM3GERAL' em C1-UFMA-ferramenta_frameworkpsi_pt-3-basica-sem-macros.xlsx: expected <class 'openpyxl.worksheet.cell_range.MultiCellRange'>
❌ Erro ao ler 'FORM3GERAL' em C1-UFR-ferramenta_frameworkpsi_pt-3-basica-sem-macros-ciclo-1_FINAL.xlsx: expected <class 'openpyxl.worksheet.cell_range.MultiCellRange'>
❌ Erro ao ler 'FORM3GERAL' em C2-FUNDAJ-ferramenta_framework_ppsi_v4_ciclo_2_FUNDAJ.xlsx: expected <class 'openpyxl.worksheet.cell_range.MultiCellRange'>
❌ Erro ao ler 'FORM3GERAL' em C2-UFMA-ferramenta_framework_ppsi_v4_ciclo_2.xlsx: expected <class 'openp

In [7]:
colunas1 = [
            'CICLO',
            'ID',
            'Medida',
            'Resposta das Medidas',
            'Encaminhamento interno',
            'Responsáveis',
            'Departamento',
            'Observação do Órgao para SGD',
            'Previsão de Início',
            'Previsão de Fim',
            'Status Plano de Ação',
            'Status Medida',
            'Nova resposta',
            'Prioridade',
#            'QTD TOTAL DE MEDIDAS',
#            'QTD MEDIDAS COM PRIORIDADE',
            'Nome da Origem.1',
            'Nome da Origem.2',
            'Ciclo_Entrega'
#            'ORGAO'
        ]
df = df_form3geral
for col in colunas1:
    if col in df.columns:
        vazios = df[col].isnull().sum() + (df[col].astype(str).str.strip() == '').sum()
        print(f"{col}: {vazios} vazios")
    else:
        print(f"Coluna '{col}' não encontrada")

CICLO: 75390 vazios
ID: 4471 vazios
Medida: 4472 vazios
Resposta das Medidas: 4472 vazios
Encaminhamento interno: 176028 vazios
Responsáveis: 154140 vazios
Departamento: 6774 vazios
Observação do Órgao para SGD: 173709 vazios
Previsão de Início: 156748 vazios
Previsão de Fim: 151991 vazios
Status Plano de Ação: 10036 vazios
Status Medida: 5034 vazios
Nova resposta: 178098 vazios
Prioridade: 4551 vazios
Nome da Origem.1: 0 vazios
Nome da Origem.2: 0 vazios
Ciclo_Entrega: 0 vazios


In [8]:
df_resumo.head(50)

,arquivo,total_linhas_no_arquivo,linhas_totais_lidas,linhas_lidas_antes,linhas_lidas_apos,total_arquivos_lidos,total_arquivos_consolidados
0,C1-ABIN-Ferramenta_frameworkpsi_pt-3-basica-se...,12736.0,310,310,310.0,615,615
1,C1-AEB-PPSI - Ciclo 1 - Framework.xlsx,12736.0,310,310,310.0,615,615
2,C1-AGU-Cópia de PPSI-AGU-Framework_Ciclo_1-em ...,12736.0,310,310,310.0,615,615
3,C1-ANA-Planilha_framework_PPSI_Frameworks.xlsx,12736.0,310,310,310.0,615,615
4,C1-ANAC-ferramenta_frameworkpsi_pt-2_02102023_...,13825.0,310,310,310.0,615,615
5,C1-ANATEL-Planilha_10914011_2023_09_06___PPSI_...,12736.0,310,310,310.0,615,615
6,C1-ANCINE-ANCINE_ferramenta_frameworkpsi_pt-3-...,12736.0,310,310,310.0,615,615
7,C1-ANEEL-PPSI-ANEEL.xlsx,12736.0,310,310,310.0,615,615
8,C1-ANM-[1]-9426969_Planilha_Planilha_PPSI_ANM_...,12736.0,310,310,310.0,615,615
9,C1-ANP-Planilha_3435981_ferramenta_frameworkps...,12736.0,310,310,310.0,615,615


In [9]:
df_form3geral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190399 entries, 0 to 190398
Data columns (total 17 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   CICLO                         190399 non-null  object
 1   ID                            190399 non-null  object
 2   Medida                        190399 non-null  object
 3   Resposta das Medidas          190399 non-null  object
 4   Encaminhamento interno        190399 non-null  object
 5   Responsáveis                  190399 non-null  object
 6   Departamento                  189111 non-null  object
 7   Observação do Órgao para SGD  190399 non-null  object
 8   Previsão de Início            190399 non-null  object
 9   Previsão de Fim               190399 non-null  object
 10  Status Plano de Ação          181574 non-null  object
 11  Status Medida                 190399 non-null  object
 12  Nova resposta                 190399 non-null  object
 13 

In [10]:
# 1) Considerando NaN e strings vazias ou só espaços em branco:
df_form3geral = df_form3geral[df_form3geral['ID'].notna()                      # remove NaN
              & df_form3geral['ID'].astype(str).str.strip().ne('')]  # remove '' ou '   '

In [11]:
df_resumo.to_excel(r"C:\Users\denis\OneDrive\Documentos\python\meuambiente\Scripts\PPSI\ANALISE_BASE\df_form3geral_resumo4.xlsx", sheet_name = "df_form3geral_resumo4", index=False)

In [12]:
# 1. Criar cópia de ID
df_form3geral['ID_num'] = df_form3geral['ID']

In [13]:
df_form3geral['ID_num'].head()

0    0.1
1    0.2
2    0.3
3    0.4
4    0.5
Name: ID_num, dtype: object

In [14]:

# 2. Converter a cópia para numérico, ignorando erros (valores inválidos viram NaN)
df_form3geral['ID_num'] = pd.to_numeric(df_form3geral['ID_num'], errors='coerce')

# 3. Remover linhas com valor menor que 0.1
df_form3geral = df_form3geral[df_form3geral['ID_num'] >= 0.1].copy()

In [15]:
df_form3geral.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185927 entries, 0 to 190398
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   CICLO                         185927 non-null  object 
 1   ID                            185927 non-null  object 
 2   Medida                        185927 non-null  object 
 3   Resposta das Medidas          185927 non-null  object 
 4   Encaminhamento interno        185927 non-null  object 
 5   Responsáveis                  185927 non-null  object 
 6   Departamento                  184639 non-null  object 
 7   Observação do Órgao para SGD  185927 non-null  object 
 8   Previsão de Início            185927 non-null  object 
 9   Previsão de Fim               185927 non-null  object 
 10  Status Plano de Ação          177193 non-null  object 
 11  Status Medida                 185927 non-null  object 
 12  Nova resposta                 185927 non-null  ob

In [16]:
# Salva em Excel
df_form3geral.to_excel(r"C:\Users\denis\OneDrive\Documentos\python\meuambiente\Scripts\PPSI\ANALISE_BASE\df_form3geral_contagem.xlsx", sheet_name = "df_form3geral_contagem", index=False)

In [17]:
df_consolidado = df_form3geral

In [18]:
df_consolidado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185927 entries, 0 to 190398
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   CICLO                         185927 non-null  object 
 1   ID                            185927 non-null  object 
 2   Medida                        185927 non-null  object 
 3   Resposta das Medidas          185927 non-null  object 
 4   Encaminhamento interno        185927 non-null  object 
 5   Responsáveis                  185927 non-null  object 
 6   Departamento                  184639 non-null  object 
 7   Observação do Órgao para SGD  185927 non-null  object 
 8   Previsão de Início            185927 non-null  object 
 9   Previsão de Fim               185927 non-null  object 
 10  Status Plano de Ação          177193 non-null  object 
 11  Status Medida                 185927 non-null  object 
 12  Nova resposta                 185927 non-null  ob

In [19]:
df_consolidado['Ciclo_Entrega'] = df_consolidado['Ciclo_Entrega'].astype(str).str.strip().str.upper()

# Filtrar somente ciclos válidos que seguem o padrão "C" seguido de número
ciclos_ordenados = sorted(
    [c for c in df_consolidado['Ciclo_Entrega'].unique() if c.startswith('C') and c[1:].isdigit()],
    key=lambda x: int(x[1:])
)

df_resultado = pd.DataFrame()
chave = ['Nome da Origem.1', 'ID']

for i, ciclo in enumerate(ciclos_ordenados):
    ciclos_ate_agora = ciclos_ordenados[:i+1]

    df_filtrado = df_consolidado[df_consolidado['Ciclo_Entrega'].isin(ciclos_ate_agora)].copy()

    # Mapeia a ordem dos ciclos dinamicamente
    ordem_map = {c: idx for idx, c in enumerate(ciclos_ate_agora)}
    df_filtrado['ordem_ciclo'] = df_filtrado['Ciclo_Entrega'].map(ordem_map)

    # Ordena para pegar o mais recente por grupo
    df_filtrado = (
        df_filtrado
        .sort_values(chave + ['ordem_ciclo'], ascending=[True]*len(chave) + [False])
        .drop_duplicates(subset=chave, keep='first')
    )

    df_filtrado['Ciclo_Contexto'] = ciclo
    df_resultado = pd.concat([df_resultado, df_filtrado.drop(columns='ordem_ciclo')], ignore_index=True)

In [20]:
df_resultado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207883 entries, 0 to 207882
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   CICLO                         207883 non-null  object 
 1   ID                            207883 non-null  object 
 2   Medida                        207883 non-null  object 
 3   Resposta das Medidas          207883 non-null  object 
 4   Encaminhamento interno        207883 non-null  object 
 5   Responsáveis                  207883 non-null  object 
 6   Departamento                  206572 non-null  object 
 7   Observação do Órgao para SGD  207883 non-null  object 
 8   Previsão de Início            207883 non-null  object 
 9   Previsão de Fim               207883 non-null  object 
 10  Status Plano de Ação          198568 non-null  object 
 11  Status Medida                 207883 non-null  object 
 12  Nova resposta                 207883 non-nul

In [21]:
# 1. Converte colunas numéricas com vírgula decimal
#for col in df_resultado.select_dtypes(include=['float', 'int']).columns:
#    df_resultado[col] = df_resultado[col].map(lambda x: f"{x:,.2f}".replace('.', ',') if pd.notnull(x) else x)

In [22]:
# Substitui "C1" por "Ciclo 1", "C2" por "Ciclo 2", etc.
df_form3geral['Ciclo_Entrega'] = df_form3geral['Ciclo_Entrega'].str.extract(r'(\d+)')[0].astype(int).apply(lambda x: f'Ciclo {x}')

In [23]:
# Salva em Excel
df_form3geral.to_excel(r"C:\Users\denis\OneDrive\Documentos\python\meuambiente\Scripts\PPSI\FONTE_PAINEL\FORM3GERAL-Plano-Trabalho_C.xlsx", sheet_name = "FORM3GERAL-Plano-Trabalho_C", index=False)

In [24]:
# Substitui "C1" por "Ciclo 1", "C2" por "Ciclo 2", etc.
df_resultado['Ciclo_Entrega'] = df_resultado['Ciclo_Entrega'].str.extract(r'(\d+)')[0].astype(int).apply(lambda x: f'Ciclo {x}')
df_resultado['Ciclo_Contexto'] = df_resultado['Ciclo_Contexto'].str.extract(r'(\d+)')[0].astype(int).apply(lambda x: f'Ciclo {x}')

In [25]:
# Salva em Excel
df_resultado.to_excel(r"C:\Users\denis\OneDrive\Documentos\python\meuambiente\Scripts\PPSI\FONTE_PAINEL\FORM3GERAL-Plano-Trabalho_Recente.xlsx", sheet_name = "FORM3GERAL-Plano-Trabalho_Recente", index=False)

In [26]:
# Salvando o DataFrame em CSV com o separador correto e garantindo que o ID seja tratado como texto
#df_resultado.to_csv(r"C:\Projetos\PPSI\FONTE_PAINEL\FORM3GERAL-Plano-Trabalho_Recente.csv",
#    sep=';',  # Usando ponto e vírgula como separador (dependendo da configuração regional)
#    index=False,  # Não incluir o índice
#    encoding='utf-8-sig',  # Codificação para garantir compatibilidade com o Excel
#    float_format="%.2f",  # Formato de números com 2 casas decimais
#    quotechar='"',  # Envolver os campos em aspas para garantir que os valores não sejam alterados
#    quoting=1  # Forçar o uso de aspas
#)


In [27]:
dh_termino = datetime.today()
duracao = dh_termino - dh_inicio
print(dh_inicio)
print(dh_termino)
print(duracao)

2025-05-26 13:44:36.704099
2025-05-26 14:17:20.520325
0:32:43.816226


In [28]:
df_resultado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207883 entries, 0 to 207882
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   CICLO                         207883 non-null  object 
 1   ID                            207883 non-null  object 
 2   Medida                        207883 non-null  object 
 3   Resposta das Medidas          207883 non-null  object 
 4   Encaminhamento interno        207883 non-null  object 
 5   Responsáveis                  207883 non-null  object 
 6   Departamento                  206572 non-null  object 
 7   Observação do Órgao para SGD  207883 non-null  object 
 8   Previsão de Início            207883 non-null  object 
 9   Previsão de Fim               207883 non-null  object 
 10  Status Plano de Ação          198568 non-null  object 
 11  Status Medida                 207883 non-null  object 
 12  Nova resposta                 207883 non-nul

In [29]:
num_duplicadas = df_resultado.duplicated().sum()
print(f"Total de linhas duplicadas: {num_duplicadas}")

Total de linhas duplicadas: 0


In [30]:
colunas1 = [
            'CICLO',
            'ID',
            'Medida',
            'Resposta das Medidas',
            'Encaminhamento interno',
            'Responsáveis',
            'Departamento',
            'Observação do Órgao para SGD',
            'Previsão de Início',
            'Previsão de Fim',
            'Status Plano de Ação',
            'Status Medida',
            'Nova resposta',
            'Prioridade',
#            'QTD TOTAL DE MEDIDAS',
#            'QTD MEDIDAS COM PRIORIDADE',
            'Nome da Origem.1',
            'Nome da Origem.2',
            'Ciclo_Entrega'
#            'ORGAO'
        ]
df = df_resultado
for col in colunas1:
    if col in df.columns:
        vazios = df[col].isnull().sum() + (df[col].astype(str).str.strip() == '').sum()
        print(f"{col}: {vazios} vazios")
    else:
        print(f"Coluna '{col}' não encontrada")

CICLO: 87294 vazios
ID: 0 vazios
Medida: 0 vazios
Resposta das Medidas: 0 vazios
Encaminhamento interno: 192099 vazios
Responsáveis: 168891 vazios
Departamento: 2846 vazios
Observação do Órgao para SGD: 190164 vazios
Previsão de Início: 171264 vazios
Previsão de Fim: 166193 vazios
Status Plano de Ação: 10560 vazios
Status Medida: 613 vazios
Nova resposta: 193586 vazios
Prioridade: 100 vazios
Nome da Origem.1: 0 vazios
Nome da Origem.2: 0 vazios
Ciclo_Entrega: 0 vazios
